# Create db

In [1]:
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.docstore.document import Document
from langchain.chains import RetrievalQA
from bs4 import BeautifulSoup

In [2]:
embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2', model_kwargs={'device': 'cpu'})
#embeddings = HuggingFaceEmbeddings(model_name='dangvantuan/sentence-camembert-large', model_kwargs={'device': 'cpu'})

C:\Users\MahamanSanoussiYahay\anaconda3\envs\RAG\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
#from sentence_transformers import SentenceTransformer
#embeddings =  SentenceTransformer("dangvantuan/sentence-camembert-large")

In [4]:
def _preprocess_text(text: str) -> str:
        soup = BeautifulSoup(text, "html.parser").text
        soup_tab = soup.expandtabs()
        cleaned_text = ""

        for line in soup_tab.split("\n"):
            if line:
                cleaned_text += ". " + line

        return cleaned_text

In [5]:
def create_chunks(text, chunk_size=1000):
    # Create Document object from text documents 
    docs = [Document(page_content=post) for post in [text]]

    # Split documents into chunks 

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size, chunk_overlap=10, separators=['\n\n', '\n', '.']
    )
    docs = text_splitter.split_documents(docs)
    return docs

## Store Embedings

In [6]:
def store_embeddings(docs):
    db = FAISS.from_documents(docs, embeddings)
    return db

# Récupération du modèle

In [7]:
from langchain.llms import CTransformers
llm = CTransformers(model="models/llama-2-7b-chat.ggmlv3.q8_0.bin",
                    model_type='llama',
                    config={'max_new_tokens': 300,
                            'temperature': 0.01}
                   )

# Définition des fonctions

In [8]:
def build_qa_system(text, prompt):
    chunk_size=400
    vector_count=2
    #cleaned_text = _preprocess_text(text)
    docs = create_chunks(text, chunk_size)
    db = store_embeddings(docs)
    qa = RetrievalQA.from_chain_type(llm=llm,
                                       chain_type='stuff',
                                       retriever=db.as_retriever(search_kwargs={'k': vector_count}),
                                       return_source_documents=True,
                                       chain_type_kwargs={'prompt': prompt}
                                      )
    return qa

In [10]:
def _post_processing(qa_answer):
    if qa_answer.startswith("Réponse : "):
        return qa_answer.split("Réponse : ")[1].qa_answer.replace("</answer>", "").replace("</s>  ", "")
    elif qa_answer.startswith("Votre réponse : "):
        return qa_answer.split("Votre réponse : ")[1].qa_answer.replace("</answer>", "").replace("</s>  ", "")
    elif qa_answer.startswith("Answer : "):
        return qa_answer.split("Answer : ")[1].replace("</answer>", "").replace("</s>  ", "")
    else:
        return qa_answer.replace("</answer>", "").replace("</s>  ", "")

In [11]:
def data_extraction(qa, query):
    res = qa({'query': query})
    answer = _post_processing(res["result"])
    print(answer)
    return answer

In [12]:
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chains.combine_documents.stuff import StuffDocumentsChain

def explication(text, prompt_personna_template):
    prompt = PromptTemplate.from_template(prompt_personna_template)
    llm_chain = LLMChain(llm=llm, prompt=prompt)
    
    docs = [Document(page_content=post) for post in [text]]
    
    stuff_chain = StuffDocumentsChain(llm_chain=llm_chain, document_variable_name="text")
    
    synthesis = stuff_chain.run(docs)
    
    return synthesis

In [13]:
#import tiktoken

In [14]:
from langchain.chains.summarize import load_summarize_chain

def summarize(text, template_resume):
    prompt = PromptTemplate(template=template_resume, input_variables=['text'])
    docs = [Document(page_content=post) for post in [text]]
    
    chain = load_summarize_chain(llm, chain_type="map_reduce", map_prompt=prompt, combine_prompt=prompt, verbose=verbose)
    
    # chain = load_summarize_chain(llm, chain_type="stuff", prompt=prompt, verbose=True)
    resume = chain.run(docs)
    return resume

In [15]:
from transformers import pipeline

"""analyzer = pipeline(
    task='text-classification',
    model="cmarkea/distilcamembert-base-sentiment",
    tokenizer="cmarkea/distilcamembert-base-sentiment"
)"""
classification = pipeline('sentiment-analysis')

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [38]:
def is_positif(text):
    result = classification(text)
    
    if result[0]['label'] == 'POSITIVE':
        return True
    else:
        return False

In [16]:
def extract_decision_component(input_string, start_line, end_line=None):
    start_index = input_string.find(start_line)
    if start_index == -1:
        return "Start line not found in the input string."

    start_index += len(start_line)

    if end_line is not None:
        end_index = input_string.find(end_line, start_index)
        if end_index == -1:
            return "End line not found in the input string after the start line."
    else:
        end_index = len(input_string)

    return input_string[start_index:end_index].strip()

In [48]:
from deep_translator import GoogleTranslator
def translate(text, target='english'):
    """
    fonction permettant de traduire un texte
    """
    return GoogleTranslator(source='auto', target=target).translate(text)

# Jeu de données

In [33]:
text_decision = """

TRIBUNAL ADMINISTRATIF
DE NANTES


N°1804534, 1804538
___________

M. et Mme G... 
et M. et Mme B...
___________

Mme Le Lay
Rapporteure
___________

Mme Piltant
Rapporteur public
___________

Audience du 4 février 2021
Décision du 25 février 2021
___________	




RÉPUBLIQUE FRANÇAISE



AU NOM DU PEUPLE FRANÇAIS



Le Tribunal administratif de Nantes

(6ème chambre)

	
68-03
54-01-04-01
C

Vu la procédure suivante :

I. Par une requête enregistrée le 17 mai 2018, sous le n° 1804534, M. et Mme C... G... et M. et Mme D... B..., représentés par de Baynast, demandent au tribunal :

1°) d’annuler l’arrêté du 4 avril 2018 par lequel le maire de Saint-Vincent-sur-Graon a autorisé M. et Mme F... à construire une maison individuelle sur un terrain situé route de la Blanchardière ;

2°) de mettre à la charge de la commune de Saint-Vincent-sur-Graon une somme de 2 500 euros au titre de l’article L. 761-1 du code de justice administrative.

Ils soutiennent que :
- le dossier de demande de permis de construire est entaché d’insuffisances ;
- l’arrêté attaqué est intervenu en méconnaissance de l’article R. 423-53 du code de l’urbanisme ;
- il est intervenu en méconnaissance de l’article L. 113-2 du code de la voirie routière ;
- il méconnait les articles AU1 et AU 2 du règlement du plan local d’urbanisme ;
- il méconnait l’article AU 3 de ce règlement ;
- il méconnait l’article R. 111-27 du code de l’urbanisme.

Par des actes enregistrés les 25 mai et 4 juillet 2018, M. et Mme D... B... déclarent se désister de leur requête.

Par une ordonnance du 18 avril 2019, le président de la formation de jugement a décidé, en application de l’article R. 611-8 du code de justice administrative, qu’il n’y avait pas lieu d’instruire cette requête.


II. Par une requête et un mémoire, enregistrés le 18 mai 2018 et le 14 octobre 2020, sous le n° 1804538, M. et Mme C... G..., représentés par Me de Baynast, demandent au tribunal :

1°) d’annuler l’arrêté du 4 avril 2018 par lequel le maire de Saint-Vincent-sur-Graon a autorisé M. et Mme F... à construire une maison individuelle sur un terrain situé route de la Blanchardière ;

2°) de mettre à la charge de la commune de Saint-Vincent-sur-Graon une somme de 2 500 euros au titre de l’article L. 761-1 du code de justice administrative.

Ils soutiennent que :
- le dossier de demande de permis de construire est entaché d’insuffisances ;
- l’arrêté attaqué est intervenu en méconnaissance de l’article R. 423-53 du code de l’urbanisme ;
- il est intervenu en méconnaissance de l’article L. 113-2 du code de la voirie routière ;
- il méconnait les articles AU1 et AU 2 du règlement du plan local d’urbanisme ;
- il méconnait l’article AU 3 de ce règlement ;
- il méconnait l’article R. 111-27 du code de l’urbanisme et l’article AU 11 du règlement du plan local d’urbanisme.


Par des mémoires en défense, enregistrés les 12 mars 2019 et 27 novembre 2020, la commune de Saint-Vincent-sur-Graon, représentée par Me Tertrais, conclut au rejet de la requête et à ce que la somme de 5 000 euros soit mise à la charge de M. et Mme G... en application de l’article L. 761-1 du code de justice administrative.

Elle fait valoir que :
- la requête est irrecevable, faute pour M. et Mme G... de justifier d’un intérêt à agir suffisant ;
- le moyen tiré de la méconnaissance de l’article R. 423-53 du code de l’urbanisme est inopérant ;
- les autres moyens soulevés pour M. et Mme G... ne sont pas fondés.


Par un mémoire enregistré le 11 décembre 2020, M. et Mme E... et A... F..., représentés par la SCP Cirier et associés, concluent au rejet de la requête et à ce que la somme de 5 000 euros soit mise à la charge de M. et Mme G... en application de l’article L. 761-1 du code de justice administrative.

Ils font valoir que :
- la requête est irrecevable, faute pour M. et Mme G... de justifier d’un intérêt à agir suffisant ;
- le moyen tiré de la méconnaissance de l’article L. 113-2 du code de la voirie routière est inopérant ;
- les autres moyens soulevés pour M. et Mme G... ne sont pas fondés.


Vu les pièces des dossiers.

Vu :
-	le code de l’urbanisme ;
-	le code de justice administrative.


Les parties ont été régulièrement averties du jour de l’audience.

Ont été entendus au cours de l’audience publique :
- le rapport de Mme Le Lay,
- les conclusions de Mme Piltant, rapporteur public,
- les observations de Me Lenfant, substituant Me de Baynast, avocat des requérants, et celles de Me Tertrais, avocat de la commune de Saint-Vincent-sur-Graon.


Considérant ce qui suit :

1.  Les requêtes n° 1804534 et 1804538 présentées pour M. et Mme G... et M. et Mme B... présentent à juger des questions semblables. Il y a lieu de les joindre pour statuer par un seul jugement.

2. Par arrêté du 4 avril 2018, le maire de Saint-Vincent-sur-Graon a délivré à M. et Mme F... un permis de construire pour la réalisation, sur la parcelle 270 B 677 située route de la Blanchardière, d’une maison individuelle portant création d’une surface de plancher de 117,15 m². Par leurs requêtes, M. et Mme G... ainsi que M. et Mme B... ont demandé l’annulation de cet arrêté.


Sur le désistement de M. et Mme B... :

3. Le désistement d’instance de M. et Mme B... est pur et simple. Rien ne s’oppose à ce qu’il en soit donné acte.


Sur la recevabilité des conclusions présentées pour M. et Mme G... : 

4. Aux termes de l’article L. 600-1-2 du code de l’urbanisme, dans sa rédaction applicable au litige : « Une personne autre que l'Etat, les collectivités territoriales ou leurs groupements ou une association n'est recevable à former un recours pour excès de pouvoir contre un permis de construire, de démolir ou d'aménager que si la construction, l'aménagement ou les travaux sont de nature à affecter directement les conditions d'occupation, d'utilisation ou de jouissance du bien qu'elle détient ou occupe régulièrement ou pour lequel elle bénéficie d'une promesse de vente, de bail, ou d'un contrat préliminaire mentionné à l'article L. 261-15 du code de la construction et de l'habitation ». » Il résulte de cet article qu’il appartient, en particulier, à tout requérant qui saisit le juge administratif d'un recours pour excès de pouvoir tendant à l’annulation d'un permis de construire, de démolir ou d'aménager, de préciser l’atteinte qu’il invoque pour justifier d’un intérêt lui donnant qualité pour agir, en faisant état de tous éléments suffisamment précis et étayés de nature à établir que cette atteinte est susceptible d’affecter directement les conditions d’occupation, d’utilisation ou de jouissance de son bien. Il appartient au défendeur, s’il entend contester l’intérêt à agir du requérant, d’apporter tous éléments de nature à établir que les atteintes alléguées sont dépourvues de réalité. Le juge de l'excès de pouvoir apprécie la recevabilité de la requête au vu des éléments ainsi versés au dossier par les parties, en écartant le cas échéant les allégations qu’il jugerait insuffisamment étayées mais sans pour autant exiger de l’auteur du recours qu’il apporte la preuve du caractère certain des atteintes qu’il invoque au soutien de la recevabilité de celui-ci. Eu égard à sa situation particulière, le voisin immédiat justifie, en principe, d'un intérêt à agir lorsqu’il fait état devant le juge, qui statue au vu de l’ensemble des pièces du dossier, d’éléments relatifs à la nature, à l’importance ou à la localisation du projet de construction.

5. Pour justifier de leur intérêt leur donnant qualité pour agir, M. et Mme G... soutiennent que leur propriété est située à proximité immédiate du projet litigieux qui, par l’édification d’une maison d’habitation sur une parcelle vierge de toute construction, va affecter la vue totalement dégagée dont ils bénéficient sur la campagne, depuis le premier étage de leur propriété. Ainsi que le font valoir les défendeurs, il ressort, toutefois, des pièces du dossier que la propriété des requérants est située rue des Lotus, à environ 130 mètres du projet, dont elle est séparée par une route et plusieurs parcelles, notamment le cimetière du village. M. et Mme G... ne sont ainsi pas fondés à se prévaloir de la qualité de voisins immédiats. Il est, en outre, constant que le projet autorisé consiste en la construction d’une maison individuelle de plain-pied d’une superficie d’environ 110 m² et d’une hauteur au faitage inférieure à trois mètres. Il ressort, en outre, des pièces du dossier que la haie boisée bordant le cimetière forme un écran végétal entre le terrain d’assiette du projet et la propriété des requérants. Ainsi, et à supposer même que la construction autorisée soit partiellement visible depuis leur maison, M. et Mme G... ne sont pas fondés à soutenir, eu égard à sa localisation et à sa faible importance, que le projet litigieux serait susceptible d’affecter directement les conditions d’occupation, d’utilisation ou de jouissance de leur bien. Par suite, ils ne justifient pas de leur intérêt à agir à l’encontre du permis de construire accordé à M. et Mme F... par l’arrêté du 4 avril 2018. Ainsi, la fin de non-recevoir opposée par les défendeurs doit être accueillie.

6. Il résulte de ce de ce qui vient d’être dit que les conclusions en annulation présentées pour M. et Mme G... doivent être rejetées. 


Sur les frais liés au litige :

7. Les dispositions de l’article L. 761-1 du code de justice administrative font obstacle à ce que la commune de Saint-Vincent-sur-Graon, qui n’a pas la qualité de partie perdante, verse à M. et Mme G... une somme au titre des frais exposés par eux et non compris dans les dépens. Il y a lieu, dans les circonstances de l’espèce, de mettre à la charge de ces derniers une somme de 750 euros au titre des frais exposés par la commune de Saint-Vincent-sur-Graon et par M. et Mme F.... 

D E C I D E :

Article 1er : Il est donné acte du désistement d’instance de M. et Mme D... B....

Article 2 : Les conclusions présentées par M. et Mme G... dans le dossier n° 1804534 et leur requête enregistrée sous le n° 1804538 sont rejetées.

Article 3 : M. et Mme G... verseront une somme de 750 (sept cent cinquante) euros à la commune de Saint-Vincent-sur-Graon et la même somme à M. et Mme F... en application de l’article L. 761-1 du code de justice administrative.

Article 4 : Le présent jugement sera notifié à M. et Mme C... G..., à M. et Mme D... B..., à la commune de Saint-Vincent-sur-Graon et à M. et Mme E... et A... F....


Délibéré après l’audience du 4 février 2021, à laquelle siégeaient :

M. Berthet-Fouqué, président,
Mme Le Lay, première conseillère,
Mme René, conseillère.

Rendu public par mise à disposition au greffe, le 25 février 2021.


La rapporteure,





Y. LE LAY

	
Le président,





J. BERTHET-FOUQUÉ

La greffière,





A.-L. LE GOUALLEC

La République mande et ordonne au préfet de la Vendée, en ce qui le concerne ou à tous huissiers de justice à ce requis en ce qui concerne les voies de droit commun contre les parties privées, de pourvoir à l’exécution de la présente décision.
Pour expédition conforme,
La greffière,

"""

In [20]:
start_line = "FRENCH REPUBLIC"
end_line = "On the merits of the judgment:"
text_litige = extract_decision_component(text_decision, start_line, end_line)

In [24]:
#text_litige

In [21]:
start_line = "On the merits of the judgment:"
end_line = "D E C I D E :"
text_motif = extract_decision_component(text_decision, start_line, end_line)

In [22]:
start_line = "D E C I D E :"
end_line = None
text_decide = extract_decision_component(text_decision, start_line, end_line)

# Prompt

In [27]:
from langchain.prompts import PromptTemplate

qa_template = """<s>[INST] <<SYS>> you are question answering system <</SYS>>
Context : {context}

Answer the following question based on the context: {question}[\INST]
"""

qa_prompt = PromptTemplate(template=qa_template,
                            input_variables=['context', 'question'])

In [62]:
#___prompt explication______

template_explication = """
Expliquer dans un langage simple le texte suivant

Contexte : {text}

L'explication doit contenit au maximum 5 lignes
"""

In [63]:
#___prompt Titre______

template_titre = """Donner un titre au texte suivant :
Contexte : {text}
UNE SEULE LIGNE
"""

In [64]:
#___prompt Resume______

template_resume = """
Veuillez fournir un résumé succinct et précis de la décision judiciaire suivante

Contexte : {text}

Le resumé doit contenir au maximum 5 lignes
"""

prompt_resume = PromptTemplate(template=template_resume, input_variables=['context'])

In [65]:
#___prompt Patrice______
# Le texte suivant est extrait d'une décision de justice. 

template_patrice = """Expliquez dans un paragraphe concis le texte suivant à Patrice agé 60 ans, dirigeant d'une PME sans expertise juridique ni expérience contentieuse:
"{text}"
L'explication doit s'adresser directement à Patrice
UNE EXPLICATION CONCISE:"""

#___prompt Anna______

template_anna = """Expliquez dans un paragraphe concis le texte suivant à Hugo, âgé de 20 ans, qui souffre de déficiences cognitives. Son handicap ne lui permet que de lire et comprendre le français facile à lire et à comprendre FALC
"{text}"
L'explication doit s'adresser directement à Patrice
UNE EXPLICATION CONCISE:"""

#___prompt Hugo______
template_hugo = """Expliquez dans un paragraphe concis le texte suivant à Anna, âgée de 30 ans, qui est réfugiée et vit en France depuis 2022. Elle apprend le français et à un niveau de langue A2.
"{text}"
L'explication doit s'adresser directement à Patrice
UNE EXPLICATION CONCISE:"""

In [52]:
#___prompt Patrice______
# Le texte suivant est extrait d'une décision de justice. 

template_patrice = """<s>[INST] <<SYS>> you are teacher <</SYS>> In a concise paragraph, explain the following text to Patrice, aged 60, manager of a small business with no legal expertise or litigation experience:
"{text}"
"""

#___prompt Anna______

template_anna = """Expliquez dans un paragraphe concis le texte suivant à Hugo, âgé de 20 ans, qui souffre de déficiences cognitives. Son handicap ne lui permet que de lire et comprendre le français facile à lire et à comprendre FALC
"{text}"
L'explication doit s'adresser directement à Patrice
UNE EXPLICATION CONCISE:"""

#___prompt Hugo______
template_hugo = """Expliquez dans un paragraphe concis le texte suivant à Anna, âgée de 30 ans, qui est réfugiée et vit en France depuis 2022. Elle apprend le français et à un niveau de langue A2.
"{text}"
L'explication doit s'adresser directement à Patrice
UNE EXPLICATION CONCISE:"""

In [54]:
text = """
The plaintiff in this case is Mr. C....
The prefect of Haute-Garonne</answer>
The plaintiff complains about the rejection of his request for admission to legal aid and the remainder of his claims by the administrative court of Toulouse.
Yes, the plaintiff Mr. C... wants the judge to overturn a decision made by the prefect of Haute-Garonne. According to the context, Mr. C... filed a petition on December 1, 2022, and additional documents on March 24, 2023, requesting the court to review and overturn the prefect's decision rejecting his request for legal aid and adjudicate on the remainder of his claims. Therefore, the plaintiff is seeking judicial review of the prefect's decision.
Yes, the plaintiff wants the judge to award them a sum of money. In the context of the passage, the plaintiff is requesting that the State bear all costs and pay their counsel a sum of 2,000 euros based on the provisions of Articles L. 761-1 of the Administrative Justice Code and Article 37, paragraph 2 of the law of July 10, 1991, relating to legal aid. This indicates that they are seeking financial compensation from the State for their legal fees and costs associated with their case.
"""

explication(text, template_patrice)

Number of tokens (513) exceeded maximum context length (512).
Number of tokens (514) exceeded maximum context length (512).
Number of tokens (515) exceeded maximum context length (512).
Number of tokens (516) exceeded maximum context length (512).
Number of tokens (517) exceeded maximum context length (512).
Number of tokens (518) exceeded maximum context length (512).
Number of tokens (519) exceeded maximum context length (512).
Number of tokens (520) exceeded maximum context length (512).
Number of tokens (521) exceeded maximum context length (512).
Number of tokens (522) exceeded maximum context length (512).
Number of tokens (523) exceeded maximum context length (512).
Number of tokens (524) exceeded maximum context length (512).
Number of tokens (525) exceeded maximum context length (512).
Number of tokens (526) exceeded maximum context length (512).
Number of tokens (527) exceeded maximum context length (512).
Number of tokens (528) exceeded maximum context length (512).
Number o

Number of tokens (646) exceeded maximum context length (512).
Number of tokens (647) exceeded maximum context length (512).
Number of tokens (648) exceeded maximum context length (512).
Number of tokens (649) exceeded maximum context length (512).
Number of tokens (650) exceeded maximum context length (512).
Number of tokens (651) exceeded maximum context length (512).
Number of tokens (652) exceeded maximum context length (512).
Number of tokens (653) exceeded maximum context length (512).
Number of tokens (654) exceeded maximum context length (512).
Number of tokens (655) exceeded maximum context length (512).
Number of tokens (656) exceeded maximum context length (512).
Number of tokens (657) exceeded maximum context length (512).
Number of tokens (658) exceeded maximum context length (512).
Number of tokens (659) exceeded maximum context length (512).
Number of tokens (660) exceeded maximum context length (512).
Number of tokens (661) exceeded maximum context length (512).
Number o

'</answer>\n\nAs a teacher, can you explain this text to Patrice in a way that is easy to understand?</s>  Of course! Here\'s an explanation of the text that should be easy to understand for Patrice:\n\nHello Patrice! In this text, we\'re talking about a legal case involving someone named Mr. C.... He\'s asking a judge to look at a decision made by a person called the "prefect" in Haute-Garonne (a region in France). Mr. C... wants the judge to overturn the prefect\'s decision because he didn\'t get the help he needed from the court. He wants the legal system.\ncourt. He wants the court. He wants the court. He wants the court. He wants the court. He wants the court. He wants the court. Mr. He wants the court. He wants the court. He wants the court. He\'legal aid office of legal system. He wants the court. He wants the court.\ncourt. He wants the court. He wants the court. He wants the court. He wants the court. He wants the court. He wants the court. Mr. He wants the court. He wants the

In [34]:
data = {
    "decision_complete": text_decision,
    "partie_litige": text_litige,
    "partie_motif": text_motif,
    "partie_resultat": text_decide
    
}

# Initialisation de la synthèse

In [67]:
synthesis = {
    "litige":
    {
        "quel est le requérant ?": "",
        "Qui a produit un mémoire en défense?": "",
        "De quoi le requérant se plaint-il (sans donner les arguments)?":"",
        "Le requérant veut-il que le juge annule une décision ?": "",
        "Le requérant veut-il que le juge lui octroie une somme d’argent ?": ""
    },
    "result":
    {
        "le requérant a-t-il  eu gain de cause ?":"",
        "Des sommes ont-elles été versées? Quel montant?":""
        
    },
    "motif":
    {
        "Pourquoi le requérant a-t-il échoué/réussi/non-lieu à statuer au regard du contexte (attention aux bonnes juridictions)?":""  
    }
    
}

In [25]:
synthesis = {
    "dispute":
    {
        "who is the plaintiff?": "",
        "who filed a defense memorandum?": "",
        "what does the plaintiff complain about (without providing the arguments)?": "",
        "does the plaintiff want the judge to overturn a decision?": "",
        "does the plaintiff want the judge to award them a sum of money?": ""
    },
    "result":
    {
        "did the plaintiff prevail?": "",
        "have any amounts been paid? If so, how much?": ""
        
    },
    "reason":
    {
        "why did the plaintiff fail/succeed/non-suit in the context (pay attention to the correct jurisdictions)?": ""  
    }
    
}

# Création de la synthèse

In [28]:
# Traitement de la partie litige
qa = build_qa_system(text_litige, qa_prompt)

import time
start_time = time.time()

for query in synthesis["dispute"]:
    synthesis["dispute"][query] = data_extraction(qa, query)
    
print("--- %s seconds ---" % (time.time() - start_time))

The plaintiff in this case is Mr. C....
The prefect of Haute-Garonne</answer>
The plaintiff complains about the rejection of his request for admission to legal aid and the remainder of his claims by the administrative court of Toulouse.
Yes, the plaintiff Mr. C... wants the judge to overturn a decision made by the prefect of Haute-Garonne. According to the context, Mr. C... filed a petition on December 1, 2022, and additional documents on March 24, 2023, requesting the court to review and overturn the prefect's decision rejecting his request for legal aid and adjudicate on the remainder of his claims. Therefore, the plaintiff is seeking judicial review of the prefect's decision.
Yes, the plaintiff wants the judge to award them a sum of money. In the context of the passage, the plaintiff is requesting that the State bear all costs and pay their counsel a sum of 2,000 euros based on the provisions of Articles L. 761-1 of the Administrative Justice Code and Article 37, paragraph 2 of the 

In [35]:
# Traitement de la partie du résultat

# Création du système QA
qa = build_qa_system(data['decision_complete'], qa_prompt)

import time
start_time = time.time()

for query in synthesis["result"]:
    synthesis["result"][query] = data_extraction(qa, query)
    
print("--- %s seconds ---" % (time.time() - start_time))

</s>  Yes, the plaintiff prevailed in the case. According to the passage, it is stated that "it follows from the foregoing, without it being necessary to rule on the other grounds of the petition, that Mr. C... is justified in arguing that it was wrongly held by the judgment under appeal that the administrative court of Toulouse rejected his request." This implies that the plaintiff's argument was successful and the judgment against him was reversed.
Yes, according to the context, an amount of 1,000 euros has been paid to Mr. Brel under Article 37, paragraph 2 of the law of July 10, 1991.
--- 79.27094960212708 seconds ---


In [40]:
# Traitement de la partie du motif

chunk_size = 500
vector_count = 2
#chunk_size = 400
#vector_count = 3
#qa = build_qa_system(text_motif, chunk_size, vector_count)

#synthesis["motif"] = {}
#result = is_positif(synthesis["result"]["le requérant a-t-il  eu gain de cause ?"])
result = is_positif(synthesis["result"]["did the plaintiff prevail?"])
                                        
import time
start_time = time.time()

if result:
    query = "Why did the plantiff win"
    synthesis["reason"][query] = data_extraction(qa, query) 
else:
    query = "Why did the plaintiff lose?"
    synthesis["reason"][query] = data_extraction(qa, query)
                                        
print("--- %s seconds ---" % (time.time() - start_time))

</s>  Based on the context provided, it appears that the plaintiff won because the administrative court of Toulouse rejected the defendant's request. The judgment under appeal held that the administrative court of Toulouse wrongly held that the defendant's request was rejected. Therefore, the plaintiff's argument that the defendant's request was wrongly rejected was upheld, resulting in a win for the plaintiff.
--- 33.08984661102295 seconds ---
